# 04 — Scenario Analysis (Rate Shocks + Linearity Check)

## Objective
Run rate shock scenarios and compare:
- full reprice P&L
- DV01 linear approximation
- residual nonlinearity (convexity / curve shape effects)

### Scenarios
- Parallel: $\pm 25bp$, $\pm 50bp$
- Steepener / flattener (piecewise linear shifts)

In [1]:
import pandas as pd

from fixed_income_engine.curves import bootstrap_curve_from_bills_notes
from fixed_income_engine.portfolio import make_sample_portfolio
from fixed_income_engine.scenarios import run_rate_scenarios
from fixed_income_engine.risk import compute_portfolio_dv01

val_date = pd.Timestamp("2026-02-13")
settle = pd.Timestamp("2026-02-16")

market = pd.DataFrame([
    {"type": "bill", "maturity": pd.Timestamp("2026-02-20"), "quote": 0.0525, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2026-03-13"), "quote": 0.0520, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2026-05-15"), "quote": 0.0515, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2026-08-14"), "quote": 0.0505, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2027-02-12"), "quote": 0.0485, "day_count": "ACT/360"},
    {"type": "note", "maturity": pd.Timestamp("2028-02-15"), "quote": 0.0450, "coupon_freq": 2, "day_count": "30/360"},
    {"type": "note", "maturity": pd.Timestamp("2031-02-15"), "quote": 0.0430, "coupon_freq": 2, "day_count": "30/360"},
    {"type": "note", "maturity": pd.Timestamp("2036-02-15"), "quote": 0.0425, "coupon_freq": 2, "day_count": "30/360"},
]).sort_values("maturity").reset_index(drop=True)

curve = bootstrap_curve_from_bills_notes(market, val_date)
portfolio_df = make_sample_portfolio(n=20, val_date=val_date, seed=7)

In [2]:
per_bond_scen, scen_summary = run_rate_scenarios(curve, portfolio_df, val_date, settle)
scen_summary

,scenario,total_pnl_per_100_notional
0,PAR_-50bp_PnL,52.431716
1,PAR_-25bp_PnL,25.989558
2,PAR_+25bp_PnL,-25.545758
3,PAR_+50bp_PnL,-50.656334
4,STEEPENER_25bp_PnL,8.904997
5,FLATTENER_25bp_PnL,-8.653248


In [4]:
dv01_df = compute_portfolio_dv01(curve, portfolio_df, val_date, settle)

tmp = per_bond_scen[["bond_id", "PAR_+25bp_PnL"]].merge(dv01_df[["bond_id", "dv01"]], on="bond_id")
tmp["linear_25bp"] = 25.0 * tmp["dv01"]
tmp["nonlinear_diff"] = tmp["PAR_+25bp_PnL"] - tmp["linear_25bp"]

display(tmp.head(10), tmp["nonlinear_diff"].describe())


,bond_id,PAR_+25bp_PnL,dv01,linear_25bp,nonlinear_diff
0,BOND_000,-1.954627,-0.079067,-1.976665,0.022038
1,BOND_001,-1.454775,-0.058660,-1.466503,0.011728
2,BOND_002,-1.447413,-0.058364,-1.459100,0.011687
3,BOND_003,-1.866738,-0.075424,-1.885598,0.018860
4,BOND_004,-1.329889,-0.053561,-1.339021,0.009132
5,BOND_005,-1.733014,-0.069943,-1.748584,0.015570
6,BOND_006,-2.131881,-0.086106,-2.152641,0.020761
7,BOND_007,-0.719858,-0.028895,-0.722371,0.002513
8,BOND_008,-0.247591,-0.009915,-0.247885,0.000294
9,BOND_009,-0.973753,-0.039128,-0.978209,0.004456


count    20.000000
mean      0.010579
std       0.008128
min       0.000288
25%       0.002487
50%       0.010410
75%       0.018122
max       0.023448
Name: nonlinear_diff, dtype: float64

## Interpreting the nonlinearity residual
If $DV01$ were a perfect linear approximation, then:
$$
\Delta P(25bp) \approx 25 \cdot DV01.
$$

The residual:
$$
\text{nonlinear\_diff} = \Delta P_{\text{full}} - \Delta P_{\text{linear}}
$$
is explained primarily by:
- convexity (2nd order)
- curve interpolation effects
- uneven cashflow timing
- non-parallel components in non-parallel scenarios